In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "ENZYMES"

# *BFS Positional Encodings with Variable k-Roots in GNNs*

In [3]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"
K_VALUES = [2,3,5,8,10]

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, bfs_dim=8, k=3):
        super().__init__()
        self.bfs_proj = torch.nn.Sequential(
            torch.nn.Linear(k, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, bfs_dim)
        )
        self.conv1 = GCNConv(in_channels + bfs_dim, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, bfs_ranks, edge_index, batch):
        bfs_feat = self.bfs_proj(bfs_ranks)
        x = torch.cat([x, bfs_feat], dim=1)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def make_bfs_transform(k):
    def transform(data):
        G = to_networkx(data, to_undirected=True)
        bfs_ranks = torch.full((data.num_nodes, k), -1.0, dtype=torch.float)
        if G.number_of_nodes() > 0:
            degrees = dict(G.degree())
            top_k_nodes = sorted(degrees, key=degrees.get, reverse=True)[:k]
            for idx, root in enumerate(top_k_nodes):
                bfs_order = list(nx.bfs_tree(G, root))
                L = max(1, len(bfs_order))
                for i, node in enumerate(bfs_order):
                    bfs_ranks[node, idx] = i / L
        data.bfs = bfs_ranks
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1))  # default feature
        return data
    return transform

def run_one_k(k_roots):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for SEED in range(1, NUM_SEEDS + 1):
        set_seed(SEED)

        # Load dataset safely
        try:
            original_dataset = TUDataset(
                root='data/TUDataset',
                name=Dataset_name,
                transform=make_bfs_transform(k_roots)
            )
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return

        # Filter None entries but keep original_dataset for num_classes
        dataset = [d for d in original_dataset if d is not None]
        if len(dataset) == 0:
            print("No valid graphs in dataset")
            return

        # Shuffle dataset
        random.shuffle(dataset)

        # Ensure features exist
        if dataset[0].x is None:
            for data in dataset:
                data.x = torch.ones((data.num_nodes, 1))

        # Get num_classes from the original dataset
        num_classes = original_dataset.num_classes
        if num_classes is None:
            num_classes = int(max([d.y.max().item() for d in dataset]) + 1)

        num_graphs = len(dataset)
        train_size = int(0.8 * num_graphs)
        train_dataset = dataset[:train_size]
        test_dataset = dataset[train_size:]

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

        wandb.init(
            project=PROJECT,
            entity=ENTITY,
            name=f"GCN_BFS_MaxDeg_{k_roots}Roots_seed_{SEED}",
            group=f"GCN_{Dataset_name}_BFS_K_Roots",
            job_type="bfs-k-roots",
            reinit=True,
            config={
                "model": "GCN",
                "hidden_channels": 64,
                "epochs": EPOCHS,
                "lr": 0.001,
                "dataset": Dataset_name,
                "task": f"BFS_{k_roots}_Roots",
                "bfs_k_roots": k_roots,
                "bfs_root_criterion": "max_degree",
                "seed": SEED,
                "cuda_available": torch.cuda.is_available(),
                "gpu_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
                "cuda_version": torch.version.cuda if torch.cuda.is_available() else None
            }
        )

        model = GCN(
            in_channels=dataset[0].x.shape[1],
            hidden_channels=64,
            out_channels=num_classes,
            k=k_roots
        ).to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

        def train():
            model.train()
            total_loss = 0.0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                loss = F.nll_loss(out, data.y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item() * data.num_graphs
            return total_loss / len(train_dataset)

        def test():
            model.eval()
            correct = 0
            for data in test_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct += int((pred == data.y).sum())
            return correct / len(test_dataset)

        for epoch in range(1, EPOCHS + 1):
            loss = train()
            test_acc = test()
            # compute train accuracy
            model.eval()
            correct_train = 0
            for data in train_loader:
                data = data.to(device)
                out = model(data.x, data.bfs, data.edge_index, data.batch)
                pred = out.argmax(dim=1)
                correct_train += int((pred == data.y).sum())
            train_acc = correct_train / len(train_dataset)
            print(f"K={k_roots} | Seed {SEED} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
            wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

        wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for k in K_VALUES:
        run_one_k(k)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250822_085019-m76izb42
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run GCN_BFS_MaxDeg_2Roots_seed_1
wandb: ⭐️ View project at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: 🚀 View run at https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/m76izb42


K=2 | Seed 1 | Epoch 001 | Loss: 1.7960 | Test Acc: 0.1250
K=2 | Seed 1 | Epoch 002 | Loss: 1.7877 | Test Acc: 0.1333
K=2 | Seed 1 | Epoch 003 | Loss: 1.7827 | Test Acc: 0.1583
K=2 | Seed 1 | Epoch 004 | Loss: 1.7768 | Test Acc: 0.1667
K=2 | Seed 1 | Epoch 005 | Loss: 1.7709 | Test Acc: 0.1500
K=2 | Seed 1 | Epoch 006 | Loss: 1.7650 | Test Acc: 0.1750
K=2 | Seed 1 | Epoch 007 | Loss: 1.7610 | Test Acc: 0.1917
K=2 | Seed 1 | Epoch 008 | Loss: 1.7543 | Test Acc: 0.1917
K=2 | Seed 1 | Epoch 009 | Loss: 1.7516 | Test Acc: 0.1750
K=2 | Seed 1 | Epoch 010 | Loss: 1.7493 | Test Acc: 0.1833
K=2 | Seed 1 | Epoch 011 | Loss: 1.7445 | Test Acc: 0.1833
K=2 | Seed 1 | Epoch 012 | Loss: 1.7439 | Test Acc: 0.1917
K=2 | Seed 1 | Epoch 013 | Loss: 1.7386 | Test Acc: 0.1917
K=2 | Seed 1 | Epoch 014 | Loss: 1.7368 | Test Acc: 0.2000
K=2 | Seed 1 | Epoch 015 | Loss: 1.7331 | Test Acc: 0.2000
K=2 | Seed 1 | Epoch 016 | Loss: 1.7335 | Test Acc: 0.1833
K=2 | Seed 1 | Epoch 017 | Loss: 1.7301 | Test Acc: 0.21

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁
wandb:  test_acc ▁▂▂▂▃▃▃▃▃▃▄▃▄▅▅▆▄▆▅▅▅▆▆▆▆▆▆▆▆▆▇▆▇▆▆▆█▇█▇
wandb: train_acc ▁▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▅▆▇▇▇▇▇▇█▆▇█▇▇▇▇▇▇▇▇▇█▇
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.63303
wandb:  test_acc 0.28333
wandb: train_acc 0.33125
wandb: 
wandb: 🚀 View run GCN_BFS_MaxDeg_2Roots_seed_1 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/m76izb42
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_085019-m76izb42/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wo

K=2 | Seed 2 | Epoch 001 | Loss: 1.7985 | Test Acc: 0.1083
K=2 | Seed 2 | Epoch 002 | Loss: 1.7887 | Test Acc: 0.1083
K=2 | Seed 2 | Epoch 003 | Loss: 1.7846 | Test Acc: 0.1083
K=2 | Seed 2 | Epoch 004 | Loss: 1.7802 | Test Acc: 0.1167
K=2 | Seed 2 | Epoch 005 | Loss: 1.7750 | Test Acc: 0.1083
K=2 | Seed 2 | Epoch 006 | Loss: 1.7683 | Test Acc: 0.1417
K=2 | Seed 2 | Epoch 007 | Loss: 1.7614 | Test Acc: 0.1333
K=2 | Seed 2 | Epoch 008 | Loss: 1.7529 | Test Acc: 0.1250
K=2 | Seed 2 | Epoch 009 | Loss: 1.7477 | Test Acc: 0.1250
K=2 | Seed 2 | Epoch 010 | Loss: 1.7425 | Test Acc: 0.1333
K=2 | Seed 2 | Epoch 011 | Loss: 1.7384 | Test Acc: 0.1083
K=2 | Seed 2 | Epoch 012 | Loss: 1.7351 | Test Acc: 0.1750
K=2 | Seed 2 | Epoch 013 | Loss: 1.7316 | Test Acc: 0.1250
K=2 | Seed 2 | Epoch 014 | Loss: 1.7268 | Test Acc: 0.1500
K=2 | Seed 2 | Epoch 015 | Loss: 1.7261 | Test Acc: 0.1417
K=2 | Seed 2 | Epoch 016 | Loss: 1.7219 | Test Acc: 0.1417
K=2 | Seed 2 | Epoch 017 | Loss: 1.7170 | Test Acc: 0.14

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:      loss ██▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁
wandb:  test_acc ▁▁▁▁▁▃▂▂▃▁▂▄▃▃▃▆▅▅▇▆▆█▅█▇▇▇█▇▆▇▇▇▇█▇▇███
wandb: train_acc ▁▁▁▂▄▄▃▅▄▅▅▅▆▅▆▆▆▇▆▇▇▇█▇▇▇▇█▇▇▇▇▇▇███▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.63403
wandb:  test_acc 0.21667
wandb: train_acc 0.30417
wandb: 
wandb: 🚀 View run GCN_BFS_MaxDeg_2Roots_seed_2 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/wgfm3s5o
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_085036-wgfm3s5o/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wo

...

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
wandb:      loss ████▇▇▇▇▇▇▇▆▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▃▂▂▁▁
wandb:  test_acc ▁▅▂▅▅▅▆▆▆▄▄▆▆▄▄▄▃▅▅▆▄▅▄▆▆▃▅▆▄▄▄▄▆▄▆▇▆▅█▇
wandb: train_acc ▁▂▂▁▂▃▂▃▂▂▂▃▃▃▃▃▃▄▃▄▄▄▄▃▆▅▅▅▅▅▆▇▆▇▆▆▆▇██
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.53036
wandb:  test_acc 0.225
wandb: train_acc 0.3875
wandb: 
wandb: 🚀 View run GCN_BFS_MaxDeg_10Roots_seed_28 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/y5rao4co
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093709-y5rao4co/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wor

K=10 | Seed 29 | Epoch 001 | Loss: 1.7950 | Test Acc: 0.1333
K=10 | Seed 29 | Epoch 002 | Loss: 1.7868 | Test Acc: 0.0917
K=10 | Seed 29 | Epoch 003 | Loss: 1.7815 | Test Acc: 0.1583
K=10 | Seed 29 | Epoch 004 | Loss: 1.7759 | Test Acc: 0.1083
K=10 | Seed 29 | Epoch 005 | Loss: 1.7681 | Test Acc: 0.1250
K=10 | Seed 29 | Epoch 006 | Loss: 1.7627 | Test Acc: 0.1750
K=10 | Seed 29 | Epoch 007 | Loss: 1.7562 | Test Acc: 0.1917
K=10 | Seed 29 | Epoch 008 | Loss: 1.7497 | Test Acc: 0.1750
K=10 | Seed 29 | Epoch 009 | Loss: 1.7432 | Test Acc: 0.1833
K=10 | Seed 29 | Epoch 010 | Loss: 1.7419 | Test Acc: 0.1667
K=10 | Seed 29 | Epoch 011 | Loss: 1.7372 | Test Acc: 0.1833
K=10 | Seed 29 | Epoch 012 | Loss: 1.7329 | Test Acc: 0.1333
K=10 | Seed 29 | Epoch 013 | Loss: 1.7314 | Test Acc: 0.2083
K=10 | Seed 29 | Epoch 014 | Loss: 1.7277 | Test Acc: 0.1917
K=10 | Seed 29 | Epoch 015 | Loss: 1.7234 | Test Acc: 0.1333
K=10 | Seed 29 | Epoch 016 | Loss: 1.7193 | Test Acc: 0.2000
K=10 | Seed 29 | Epoch 0

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁
wandb:  test_acc ▃▁▄▂▂▅▅▅▄▅▆▅▃▆▆▅▅▅▆▆▆▇▆▅▅▆▆▇▆▅▆▅▇▅▇▇▆▆█▇
wandb: train_acc ▁▁▃▂▂▃▄▄▃▃▄▄▃▅▅▅▅▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇█
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.53833
wandb:  test_acc 0.225
wandb: train_acc 0.36875
wandb: 
wandb: 🚀 View run GCN_BFS_MaxDeg_10Roots_seed_29 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/h3mtfqep
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093730-h3mtfqep/logs
wandb: Tracking run with wandb version 0.20.1
wandb: Run data is saved locally in /kaggle/wo

K=10 | Seed 30 | Epoch 001 | Loss: 1.7999 | Test Acc: 0.2000
K=10 | Seed 30 | Epoch 002 | Loss: 1.7901 | Test Acc: 0.1500
K=10 | Seed 30 | Epoch 003 | Loss: 1.7865 | Test Acc: 0.1667
K=10 | Seed 30 | Epoch 004 | Loss: 1.7804 | Test Acc: 0.1250
K=10 | Seed 30 | Epoch 005 | Loss: 1.7770 | Test Acc: 0.1333
K=10 | Seed 30 | Epoch 006 | Loss: 1.7697 | Test Acc: 0.1750
K=10 | Seed 30 | Epoch 007 | Loss: 1.7627 | Test Acc: 0.1833
K=10 | Seed 30 | Epoch 008 | Loss: 1.7537 | Test Acc: 0.1417
K=10 | Seed 30 | Epoch 009 | Loss: 1.7471 | Test Acc: 0.1667
K=10 | Seed 30 | Epoch 010 | Loss: 1.7413 | Test Acc: 0.1583
K=10 | Seed 30 | Epoch 011 | Loss: 1.7374 | Test Acc: 0.1833
K=10 | Seed 30 | Epoch 012 | Loss: 1.7345 | Test Acc: 0.1667
K=10 | Seed 30 | Epoch 013 | Loss: 1.7289 | Test Acc: 0.2333
K=10 | Seed 30 | Epoch 014 | Loss: 1.7247 | Test Acc: 0.1667
K=10 | Seed 30 | Epoch 015 | Loss: 1.7177 | Test Acc: 0.1667
K=10 | Seed 30 | Epoch 016 | Loss: 1.7135 | Test Acc: 0.2417
K=10 | Seed 30 | Epoch 0

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:     epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
wandb:      loss ███▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁
wandb:  test_acc ▅▂▃▁▁▄▂▃▃▄▇▃█▆▃█▄▃▇▄▄▆█▇▅▄█▅▆▆▅▆▅▇▆▆▅▇▄▆
wandb: train_acc ▁▃▄▃▂▄▃▄▄▃▅▄▅▆▅▆▅▆▅▆▆▅█▆▆▆▆█▇▇▇▇██▇█████
wandb: 
wandb: Run summary:
wandb:     epoch 50
wandb:      loss 1.59644
wandb:  test_acc 0.21667
wandb: train_acc 0.325
wandb: 
wandb: 🚀 View run GCN_BFS_MaxDeg_10Roots_seed_30 at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal/runs/38du1bfy
wandb: ⭐️ View project at: https://wandb.ai/noynetanel1-tel-aviv-university/GCN%20Graph%20classification%28ENZYMES%29_Gal
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250822_093752-38du1bfy/logs
